In [1]:
from MesoPy import Meso
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import json
import ast
import pandas as pd
from datetime import datetime, timedelta
# OS interaction
import sys
import os

In [2]:
# Dictonary of Column Names - Map MesoNet Names to Standard Names

pa_cw = {
    'air_temp_set_1_amin': 'tn',
    'air_temp_set_1_amax': 'tx',
    'relative_humidity_set_1_mean': 'rh',
    'dew_point_temperature_set_1_mean': 'td',
    'wind_speed_set_1_median': 'ff',
    'solar_radiation_set_1_sum': 'sr',
    'PP': 'rr'
}

# List of MesoWest Variables

var_list = ['air_temp_set_1', 
            'dew_point_temperature_set_1', 
            'relative_humidity_set_1', 
            'wind_speed_set_1',
            'wind_direction_set_1',
            'solar_radiation_set_1'] 

# Parameter Ranges for Quality Control (QC)

pa_qc = pd.DataFrame(index=['Min', 'Max'], columns=var_list)
pa_qc['air_temp_set_1_amin'] = np.array([-100,150])
pa_qc['air_temp_set_1_amax'] = np.array([-100,150])
pa_qc['relative_humidity_set_1_mean'] = np.array([0,100])
pa_qc['dew_point_temperature_set_1_mean'] = np.array([-100,150])
pa_qc['wind_speed_set_1_median'] = np.array([0,150])
pa_qc['solar_radiation_set_1_sum'] = np.array([0,10000])

pa_qc.rename(columns=pa_cw, inplace=True)

In [3]:
def mesowest_api_download(token, date_start_inp, date_end_inp, sta_list):
    mso = Meso(token=token)
    date_start = date_start_inp.strftime('%Y%m%d%H%M')
    date_end = date_end_inp.strftime('%Y%m%d%H%M')
    
    var_list = ['air_temp_set_1', 'dew_point_temperature_set_1', 'relative_humidity_set_1', 'wind_speed_set_1','wind_direction_set_1','solar_radiation_set_1'] 

    date_start = date_start_inp.strftime('%Y%m%d%H%M')
    date_end = date_end_inp.strftime('%Y%m%d%H%M')

    data_json = mso.timeseries(stid=sta_list, start=date_start, end=date_end)

    # Get Station Info
    ct_sta = data_json['SUMMARY']['NUMBER_OF_OBJECTS']
    sta_info_elev = [ast.literal_eval(json.dumps(data_json['STATION'][iter_sta]['ELEVATION'])) for iter_sta in range(0,ct_sta)]
    sta_info_lat = [ast.literal_eval(json.dumps(data_json['STATION'][iter_sta]['LATITUDE'])) for iter_sta in range(0,ct_sta)]
    sta_info_lon = [ast.literal_eval(json.dumps(data_json['STATION'][iter_sta]['LONGITUDE'])) for iter_sta in range(0,ct_sta)]
    sta_info_name = [ast.literal_eval(json.dumps(data_json['STATION'][iter_sta]['NAME'])) for iter_sta in range(0,ct_sta)]
    sta_info_id = [ast.literal_eval(json.dumps(data_json['STATION'][iter_sta]['STID'])) for iter_sta in range(0,ct_sta)]

    
    # Parse Data to Pandas Dataframe
    data_sta = data_json['STATION'][0]['OBSERVATIONS']
    time_vec = [datetime.strptime(data_sta['date_time'][cd], '%Y-%m-%dT%H:%M:%SZ') for cd in range(len(data_sta['date_time']))]
    index = time_vec
    del data_sta['date_time']
    columns = list(data_sta.keys())

    data_df = pd.DataFrame(index=index, columns=columns)
    for key in data_sta:
        data_df[key] = data_sta[key]
    
    # Add in Columns for Missing Variables 
    for var in var_list:
        if var not in data_df:
            data_df[var] = np.nan
        
    # Aggregate Data to Daily
    data_agg_df = data_df.resample('1D').agg({'air_temp_set_1' : [np.min, np.max], 'dew_point_temperature_set_1' : [np.mean], 'relative_humidity_set_1' : [np.mean], 'wind_speed_set_1' : [np.median], 'solar_radiation_set_1': np.sum})
    data_agg_df.columns = ["_".join(x) for x in data_agg_df.columns.ravel()]
    return(data_agg_df)

In [11]:
date_start_inp = datetime(1997,1,1,0,0) 
date_end_inp = datetime(2018,9,30,23,0) 
token = '9f084cfd461244ae948d1c3ea4c7f0d5'
sta_hours = int((date_end_inp-date_start_inp).total_seconds()/60/ 60) 
print(sta_hours) # full request station-hours if available
# API will only allow 100,000 station-hours at a time 

stid_list = ['coam8',
 'coopdvnm8',
 'coopgltm8',
 'coopsaom8',
 'coopsdym8',
 'd5963',
 'e1880',
 'e2671',
 'e5606',
 'ggwcr',
 'ggwdo',
 'ggwnm',
 'ggwsc',
 'ggwsw',
 'ggwwr',
 'hrlm',
 'hwsm8',
 'mscm8',
 'mwsm']

190631


In [14]:
for stid_sel in stid_list:
    # API Call
    df_out = mesowest_api_download(token,date_start_inp,date_end_inp,stid_sel)
        
    # Rename Data Columns
    df_out.rename(columns=pa_cw, inplace=True)
    df_out.index.name='Date'

    # QC Check and Unit Conversion
    for col in df_out:
        df_out[col + '_QC'] = np.where((df_out[col]>= pa_qc[col][0]) & (df_out[col]<= pa_qc[col][1]), 1, 0)
        
    # Save Data to File 
    file_out = stid_sel + '.dat'
    df_out.to_csv(file_out)
